## General example
This example covers some of the core functionalities of the toolbox.
### Outline
- Load a test dataset
- Preliminary analysis
 - Show summary
 - Show some plots
- Preprocess data
- Evaluate a classifier
- Show some model selection plots

In [2]:
import sys
sys.path.append("C:\\Users\\aartetxe\\ARKAITZ\\par-toolbox\\partb")
sys.path.append("C:\\Users\\aartetxe\\ARKAITZ\\par-toolbox")

In [3]:
# Dependencies
import pandas as pd
import numpy as np
from datetime import datetime
from utility.preprocessing import PreProcessing

In [4]:
# Allow hot reload of modules
%load_ext autoreload
%autoreload 2
# Show plots within notebook
%matplotlib inline

In [54]:
# Load data
df = pd.read_csv('../data/readmission_example.csv', sep=';')

In [55]:
# Construct preprocessor
prepro = PreProcessing(df, 'OUTCOME')

# Calculate and set length of stay (LOS) 
prepro.set_los('HOSP_DATE', 'DISCHARGE_DATE')
prepro.set_first_diagnostic('FIRSTDIAG', 'HOSP_DATE')
prepro.set_age('DAY_OF_BIRTH', 'HOSP_DATE')

df.head()

,PATIENT_ID,HOSP_DATE,DISCHARGE_DATE,DAY_OF_BIRTH,SEX,SMOKE,LVEF,FIRSTDIAG,NEED_OXYGEN,UREA,CREATININE,SODIUM,POTASSIUM,HEMOGLOBIN,SINUS_RHYTHM,ATRIAL_FIBRILLATION,PACEMAKER_RHYTHM,OUTCOME,LOS,AGE
0,PATIENT001,22/02/2014,05/03/2014,01/01/1949,female,yes,20,0,0,45,0.94,139,5.30,15.3,1,0,0,readmitted,11.0,65.0
1,PATIENT002,14/03/2014,21/03/2014,01/01/1924,male,no,20,34,0,95,1.39,141,4.30,10.2,1,0,0,readmitted,7.0,90.0
2,PATIENT003,14/05/2014,16/05/2014,01/01/1947,male,yes,40,1,0,47,1.01,145,3.06,16.6,1,0,0,not-readmitted,2.0,67.0
3,PATIENT004,29/03/2014,09/04/2014,01/01/1928,male,no,25,18,0,50,1.13,145,2.94,14.0,0,0,1,readmitted,11.0,86.0
4,PATIENT005,NaN,NaN,01/01/1949,male,former,60,NaN,0,42,1.33,141,4.30,10.5,0,1,0,not-readmitted,NaN,NaN


In [56]:
# Encode strings
df['OUTCOME'] = map(lambda x: 1 if x=='readmitted' else 0, df['OUTCOME'])
df['SEX'] = map(lambda x: 1 if x=='male' else 0, df['SEX'])

def encode_smoke(x):
    if x == 'yes':
        return 2 # smoker
    elif x == 'no':
        return 0 # not smoker
    elif x == 'former':
        return 1 # former smoker
    else:
        return

df['SMOKE'] = map(encode_smoke, df['SMOKE'])

# Remove unnecesary data
del df['PATIENT_ID']
del df['HOSP_DATE']
del df['DISCHARGE_DATE']
del df['DAY_OF_BIRTH']

# Simple missing value imputation
df.fillna(0, inplace=True)

df.head()

,SEX,SMOKE,LVEF,FIRSTDIAG,NEED_OXYGEN,UREA,CREATININE,SODIUM,POTASSIUM,HEMOGLOBIN,SINUS_RHYTHM,ATRIAL_FIBRILLATION,PACEMAKER_RHYTHM,OUTCOME,LOS,AGE
0,0,2,20,0,0,45,0.94,139,5.30,15.3,1,0,0,1,11.0,65.0
1,1,0,20,34,0,95,1.39,141,4.30,10.2,1,0,0,1,7.0,90.0
2,1,2,40,1,0,47,1.01,145,3.06,16.6,1,0,0,0,2.0,67.0
3,1,0,25,18,0,50,1.13,145,2.94,14.0,0,0,1,1,11.0,86.0
4,1,1,60,0,0,42,1.33,141,4.30,10.5,0,1,0,0,0.0,0.0


In [57]:
# Split data and label
dataset = df.drop(['OUTCOME'], axis=1)
target = df['OUTCOME']
# Format dataset and label
X = dataset.as_matrix()
y = target.as_matrix().astype(int)

In [58]:
# Test a classifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

cls = LogisticRegression()

arr = []
for i in range(0,10):
    ema = cross_val_score(cls, X, y, cv=10, scoring='roc_auc')
    arr.append(np.mean(ema))

print "Mean AUC (standard deviation): %.2f (%.3f)" % (np.mean(arr), np.std(arr))

Mean AUC (standard deviation): 0.57 (0.000)
